## Recording your own live-recordings
使用 colab 錄音(預設檔名: audio_ds.wav)

In [ ]:
!pip install 

In [ ]:
from IPython.display import Audio
from scipy.io import wavfile
import numpy as np

# from https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""


def record(sec, filename="audio.wav"):
    display(Javascript(RECORD))
    s = output.eval_js("record(%d)" % (sec * 1000))
    b = b64decode(s.split(",")[1])
    with open(filename, "wb+") as f:
        f.write(b)


audio = "audio.wav"
second = 5
print(f"Speak to your microphone {second} sec...")
record(second, audio)
print("Done!")

# Plot Audio
import librosa
import librosa.display

speech, rate = librosa.load(audio, sr=16000)
librosa.display.waveplot(speech, sr=rate)

import matplotlib.pyplot as plt

plt.show()

import pysndfile

file_name = "./audio.wav"
pysndfile.sndio.write(file_name, speech, rate=rate, format="wav", enc="pcm16")

# Playing the audio file
Audio(file_name)

In [ ]:
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sample rate:',framerate,'Hz')
print('Total time:',len(sounddata)/framerate,'s')